In [ ]:
# https://towardsdatascience.com/deep-learning-vs-puzzle-games-e996feb76162abs
# https://github.com/kgaspard/flow-free-ai

In [ ]:
# solution visualizer

In [ ]:
# solution sanity checker

In [1]:
import os
import sys
import git
import pathlib

In [2]:
PROJ_ROOT_PATH = pathlib.Path(git.Repo('.', search_parent_directories=True).working_tree_dir)
PROJ_ROOT =  str(PROJ_ROOT_PATH)
if PROJ_ROOT not in sys.path:
    sys.path.append(PROJ_ROOT)
print(PROJ_ROOT)  

In [ ]:
data_folder = pathlib.Path(PROJ_ROOT, "data")
data_folder

In [7]:
filename = "five.txt"
file = pathlib.Path(data_folder, filename)

In [9]:
# removing the new line characters
with open(file) as f:
    lines = [line.rstrip() for line in f]

In [12]:
set0 = lines[0]

In [14]:
size = set0[0]
print(size)

5


In [17]:
problem, solution = set0[2:].split("=") # ignore "5;"

In [18]:
problem

'1:1;5:4;3:3;1:2;2:4;3:2;4:2;5:5'

In [19]:
solution

'1:1;2:1;3:1;4:1;5:1;5:2;5:3;5:4|3:3;3:4;3:5;2:5;1:5;1:4;1:3;1:2|2:4;2:3;2:2;3:2|4:2;4:3;4:4;4:5;5:5'

In [23]:
import re
# Group substrings based on the specified pattern and create subproblem
subproblem = re.compile(r"\d:\d;\d:\d") # search pattern: <decimal_digit:decimal_digit;decimal_digit:decimal_digit>
subproblem_keys = subproblem.findall(problem)

In [25]:
subsolutions_entries = solution.split("|") 

In [27]:
set0_dict = dict(zip(subproblem_keys, subsolutions_entries))

In [28]:
def set_check(probl_soln_set_dict: dict):
    """Check if the solution matches the problem"""
    for key, value in 
    return
    
set0_dict

{'1:1;5:4': '1:1;2:1;3:1;4:1;5:1;5:2;5:3;5:4',
 '3:3;1:2': '3:3;3:4;3:5;2:5;1:5;1:4;1:3;1:2',
 '2:4;3:2': '2:4;2:3;2:2;3:2',
 '4:2;5:5': '4:2;4:3;4:4;4:5;5:5'}